In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
rate = pd.read_csv('./ratings_small.csv')

In [3]:
rate_mtrx=rate.pivot(index='userId', columns='movieId', values='rating')
##  select the first 7 movies only
rate_mtrx=rate_mtrx.reset_index()[['userId',1,2,3,4,5,6,7]]

In [4]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(rate_mtrx, test_size=0.2)

In [5]:
total_sum = (train[1].sum(skipna=True) +
      train[2].sum(skipna=True) +
      train[3].sum(skipna=True) +
      train[4].sum(skipna=True) +
      train[5].sum(skipna=True) +
      train[6].sum(skipna=True) +
      train[7].sum(skipna=True))

total_entries = (~np.isnan(train)).sum(1).sum()

train_mean = total_sum/total_entries

print("training raw average: ", train_mean)

training raw average:  1.7935393258426966


In [6]:
### calculating rmse for training data
stan_err = []
for i in train.columns[1:8]:
    for j in train[i]:
        if  np.isnan(j) == False:
            stan_err.append(j-train_mean)
train_rmse = math.sqrt(sum(stan_err)/len(stan_err))

print("training RMSE: ", train_rmse)

training RMSE:  1.3442561452088357


In [7]:
stan_err = []
for i in test.columns[1:8]:
    for j in test[i]:
        if  np.isnan(j) == False: 
            stan_err.append(j-train_mean)

test_rmse = math.sqrt(sum(stan_err)/len(stan_err))

print("testing RMSE: ", test_rmse)

testing RMSE:  1.3381228731659942


We want to find the baseline predictor, so we need to find the movie bias for each movie and user bias for each user

In [8]:
### calculate and store movie bias for each movie
movie_bias = []
for movie in train.columns[1:8]:
    movie_bias.append(train[movie].mean(skipna=True)-train_mean)
    
print(movie_bias)

[2.0989606741573033, 1.6175717852684144, 1.4064606741573036, 0.591076058772688, 1.4964606741573034, 2.0770489094514213, 1.513278855975485]


In [20]:
### calculate and store user bias for each user
user_bias = []
for user in range(len(train)):
    user_bias.append(train.iloc[user,1:8].mean(skipna=True)-train_mean)
    
print(len(user_bias))

536


In [21]:
### create the baseline dataframe
column_names = ['1','2','3','4','5','6','7']

baseline = pd.DataFrame(columns = column_names)

for i in range(len(user_bias)):
    for j in range(len(movie_bias)):
        baseline.append(train_mean + user_bias[i] + movie_bias[j])
        
print(baseline)

TypeError: cannot concatenate object of type '<class 'numpy.float64'>'; only Series and DataFrame objs are valid